In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG_stickyActG_stickyStimGL_wslsG_hG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['selectStimIndex']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[outcomeRew for ([id],[outcomeRew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['selectStimIndex'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[2, 1, 2, 0, 2, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 1, 1, 2, 1,
           2, 2, 0, 1, 2, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0,
           0, 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=3, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/fullModel_selectedStimulus/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:28:05,495 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:28:05,496 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:28:05,498 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:28:05,499 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:28:05,501 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:28:20,482 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/fullModel_selectedStimulus/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:28:46,416 - DeepRL - DEBUG - global iter =    0 total obj: 17933.0650
2021-07-02 14:29:05,803 - DeepRL - DEBUG - global iter =    1 total obj: 16492.8938
2021-07-02 14:29:25,722 - DeepRL - DEBUG - global iter =    2 total obj: 14483.7220
2021-07-02 14:29:47,323 - DeepRL - DEBUG - global iter =    3 total obj: 13427.0310
2021-07-02 14:30:10,685 - DeepRL - DEBUG - global iter =    4 total obj: 13034.3584
2021-07-02 14:30:34,046 - DeepRL - DEBUG - global iter =    5 total obj: 12835.8191
2021-07-02 14:30:57,310 - DeepRL - DEBUG - global iter =    6 total obj: 12688.9972
2021-07-02 14:31:22,928 - DeepRL - DEBUG - global iter =    7 total obj: 12604.1296
2021-07-02 14:31:50,897 - DeepRL - DEBUG - global iter =    8 total obj: 12521.1420
2021-07-02 14:32:18,628 - DeepRL - DEBUG - global iter =    9 total obj: 12470.9724
2021-07-02 14:32:46,502 - DeepRL - DEBUG - global iter =   10 total obj: 12428.6254
2021-07-02 14:33:15,041 - DeepRL - DEBUG - global iter =   11 total obj: 123

INFO:tensorflow:../results/fullModel_selectedStimulus/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 25,   0,  25, ...,  25,  25,  25],
          [  0, -25,   0, ...,   0,   0,   0],
          [ 25,   0,  25, ...,   0,   0,  25],
          ...,
          [  0, -25,   0, ...,   0,   0,   0],
          [ 25,  25,  25, ...,   0,  25,   0],
          [  0,   0,   0, ..., -25,   0, -25]], dtype=int16),
   'action': array([[2, 1, 2, ..., 1, 2, 2],
          [2, 1, 2, ..., 1, 2, 2],
          [0, 0, 0, ..., 0, 0, 2],
          ...,
          [2, 1, 2, ..., 1, 2, 2],
          [0, 2, 0, ..., 0, 2, 0],
          [0, 2, 0, ..., 0, 2, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
  

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:48:16,267 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:48:16,268 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:48:16,270 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:48:16,271 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:48:16,272 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:48:37,450 - DeepRL - DEBUG - opt started...
2021-07-02 14:48:37,452 - DeepRL - DEBUG - started testing...
2021-07-02 14:51:07,038 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:51:27,027 - DeepRL - DEBUG - global iter =    0 total obj: 17886.7003
2021-07-02 14:51:37,939 - DeepRL - DEBUG - global iter =    1 total obj: 15189.6453
2021-07-02 14:51:49,529 - DeepRL - DEBUG - global iter =    2 total obj: 13793.1593
2021-07-02 14:52:01,321 - DeepRL - DEBUG - global iter =    3 total obj: 13418.8219
2021-07-02 14:52:13,087 - DeepRL - DEBUG - global iter =    4 total obj: 13251.7726
2021-07-02 14:52:24,875 - DeepRL - DEBUG - global iter =    5 total obj: 13140.4284
2021-07-02 14:52:39,272 - DeepRL - DEBUG - global iter =    6 total obj: 13047.8249
2021-07-02 14:52:55,651 - DeepRL - DEBUG - global iter =    7 total obj: 12957.9650
2021-07-02 14:53:13,027 - DeepRL - DEBUG - global iter =    8 total obj: 12886.6460
2021-07-02 14:53:31,009 - DeepRL - DEBUG - global iter =    9 total obj: 12832.7639
2021-07-02 14:53:31,011 - DeepRL - DEBUG - started testing...
2021-07-02 14:55:58,323 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:56:19,624 - DeepRL - DEBUG - global iter =   10 total obj: 12786.8577
2021-07-02 14:56:30,852 - DeepRL - DEBUG - global iter =   11 total obj: 12738.6912
2021-07-02 14:56:42,631 - DeepRL - DEBUG - global iter =   12 total obj: 12681.6369
2021-07-02 14:56:54,276 - DeepRL - DEBUG - global iter =   13 total obj: 12642.7712
2021-07-02 14:57:06,770 - DeepRL - DEBUG - global iter =   14 total obj: 12601.6341
2021-07-02 14:57:21,553 - DeepRL - DEBUG - global iter =   15 total obj: 12601.7573
2021-07-02 14:57:36,797 - DeepRL - DEBUG - global iter =   16 total obj: 12556.5025
2021-07-02 14:57:54,338 - DeepRL - DEBUG - global iter =   17 total obj: 12516.7751
2021-07-02 14:58:11,301 - DeepRL - DEBUG - global iter =   18 total obj: 12486.7204
2021-07-02 14:58:28,623 - DeepRL - DEBUG - global iter =   19 total obj: 12442.5967
2021-07-02 14:58:28,625 - DeepRL - DEBUG - started testing...
2021-07-02 15:00:40,286 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/fullModel_selectedStimulus/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:05:24,270 - DeepRL - DEBUG - global iter =   30 total obj: 12162.0566
2021-07-02 15:05:37,490 - DeepRL - DEBUG - global iter =   31 total obj: 12146.8875
2021-07-02 15:05:51,851 - DeepRL - DEBUG - global iter =   32 total obj: 12138.5060
2021-07-02 15:06:06,488 - DeepRL - DEBUG - global iter =   33 total obj: 12118.4218
2021-07-02 15:06:18,941 - DeepRL - DEBUG - global iter =   34 total obj: 12119.9576
2021-07-02 15:06:32,917 - DeepRL - DEBUG - global iter =   35 total obj: 12079.2827
2021-07-02 15:06:47,282 - DeepRL - DEBUG - global iter =   36 total obj: 12068.1241
2021-07-02 15:07:01,929 - DeepRL - DEBUG - global iter =   37 total obj: 12062.1268
2021-07-02 15:07:16,302 - DeepRL - DEBUG - global iter =   38 total obj: 12066.2937
2021-07-02 15:07:28,185 - DeepRL - DEBUG - global iter =   39 total obj: 12004.7582
2021-07-02 15:07:28,186 - DeepRL - DEBUG - started testing...
2021-07-02 15:09:17,262 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_selectedStimulus/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:09:46,903 - DeepRL - DEBUG - global iter =   40 total obj: 11987.0106
2021-07-02 15:09:57,991 - DeepRL - DEBUG - global iter =   41 total obj: 11969.6481
2021-07-02 15:10:12,279 - DeepRL - DEBUG - global iter =   42 total obj: 11947.8740
2021-07-02 15:10:26,821 - DeepRL - DEBUG - global iter =   43 total obj: 11949.5259
2021-07-02 15:10:39,611 - DeepRL - DEBUG - global iter =   44 total obj: 11941.7867
2021-07-02 15:10:50,899 - DeepRL - DEBUG - global iter =   45 total obj: 11961.5935
2021-07-02 15:11:02,168 - DeepRL - DEBUG - global iter =   46 total obj: 11926.3271
2021-07-02 15:11:15,386 - DeepRL - DEBUG - global iter =   47 total obj: 11901.5391
2021-07-02 15:11:29,039 - DeepRL - DEBUG - global iter =   48 total obj: 11869.9144
2021-07-02 15:11:40,392 - DeepRL - DEBUG - global iter =   49 total obj: 11885.5883
2021-07-02 15:11:40,394 - DeepRL - DEBUG - opt finished.
2021-07-02 15:11:40,396 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:10,828 - DeepRL - DEBUG 

INFO:tensorflow:../results/fullModel_selectedStimulus/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[1, 2, 2, 1, 2, 2, 1, 1, 1, 0, 2, 1, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
          2, 2, 2, 1, 2, 0, 2, 0, 1, 1, 1, 2, 1, 2, 0, 0, 1, 0, 1, 1, 0, 0,
          2, 2, 1, 0]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2